In [1]:
import pandas as pd

In [2]:
master = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Master')
bonus = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Bonus cards')

In [3]:
master = master.drop([0, 1])

In [4]:
master['id'] = master.index
bonus['id'] = bonus.index + 1000

In [5]:
master = master[master['Expansion'] != 'chinesepromo']

In [6]:
master.to_json('master.json', orient='records', index=True)
bonus.to_json('bonus.json', orient='records', index=True)